In [36]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl

In [37]:
dfRows = pl.read_parquet("./saved_bolillas.parquet")

display(dfRows.head(10))

id,combo,b1,b2,b3,b4,b5,b6,is_rare_combination,reason,score_left_b1,score_left_b2,score_left_b3,score_left_b4,score_left_b5,score_left_b6,score_r8_b1,score_r8_b2,score_r8_b3,score_r8_b4,score_r8_b5,score_r8_b6,shared_values,shared_values_count,score_row,jumps_map
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,bool,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],u8,f64,str
"""010203040506""","[1, 2, … 6]",1,2,3,4,5,6,false,0,1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,1.0,0.0222,"[0.0222, 1.0]",2,6.1332,"""01_01_01_01_01"""
"""010203040507""","[1, 2, … 7]",1,2,3,4,5,7,true,1,1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.5,0.0222,"[0.0222, 1.0]",2,5.6332,"""01_01_01_01_02"""
"""010203040508""","[1, 2, … 8]",1,2,3,4,5,8,false,0,1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.3333,0.0222,"[0.0222, 1.0]",2,5.4665,"""01_01_01_01_03"""
"""010203040509""","[1, 2, … 9]",1,2,3,4,5,9,false,0,1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.25,0.0222,"[0.0222, 1.0]",2,5.3832,"""01_01_01_01_04"""
"""010203040510""","[1, 2, … 10]",1,2,3,4,5,10,false,0,1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.2,0.0222,"[0.0222, 1.0]",2,5.3332,"""01_01_01_01_05"""
"""010203040511""","[1, 2, … 11]",1,2,3,4,5,11,true,1,1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.1667,0.0222,"[0.0222, 1.0]",2,5.2999,"""01_01_01_01_06"""
"""010203040512""","[1, 2, … 12]",1,2,3,4,5,12,false,0,1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.1429,0.0222,"[0.0222, 1.0]",2,5.2761,"""01_01_01_01_07"""
"""010203040513""","[1, 2, … 13]",1,2,3,4,5,13,true,1,1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.125,0.0222,"[0.0222, 1.0]",2,5.2582,"""01_01_01_01_08"""
"""010203040514""","[1, 2, … 14]",1,2,3,4,5,14,false,0,1.0,0.0222,0.0222,0.0222,0.0222,0.0222,1.0,1.0,1.0,1.0,0.1111,0.0222,"[0.0222, 1.0]",2,5.2443,"""01_01_01_01_09"""


# Get random ticket

In [38]:
dfRows.sample(n=1)  

id,combo,b1,b2,b3,b4,b5,b6,is_rare_combination,reason,score_left_b1,score_left_b2,score_left_b3,score_left_b4,score_left_b5,score_left_b6,score_r8_b1,score_r8_b2,score_r8_b3,score_r8_b4,score_r8_b5,score_r8_b6,shared_values,shared_values_count,score_row,jumps_map
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,bool,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],u8,f64,str
"""072438394648""","[7, 24, … 48]",7,24,38,39,46,48,false,0,0.0435,0.0256,0.0435,0.1,0.1,0.25,0.0435,0.0278,0.0278,0.0238,0.0233,0.25,[],0,0.9588,"""17_14_01_07_02"""


# ANALYZE

In [39]:
dfRows.group_by(
  'reason'
).agg(pl.count('reason').alias('count'))

reason,count
i64,u32
0,14470411
1,835284


In [40]:
def show_b_stats (col: str):
  df_viz = dfRows.group_by(
    col
  ).agg(pl.count(col).alias('count'))

  chart = df_viz.plot.bar(
    x=col,
    y='count',
    color=col,
  )

  display(df_viz)
  display(chart)


In [42]:
show_b_stats('score_left_b2')
show_b_stats('score_r8_b2')


score_left_b2,count
f64,u32
0.125,781
0.025,1043236
0.0588,19914
0.0233,1479260
0.0333,267623
…,…
0.0909,2948
0.0385,137857
0.0526,32880


alt.Chart(...)

score_r8_b2,count
f64,u32
0.0769,573931
0.0476,642862
0.0333,368342
0.025,44909
0.125,345464
…,…
0.1667,234951
0.0909,494960
0.0556,664974


alt.Chart(...)

In [44]:
show_b_stats('score_left_b4')
show_b_stats('score_r8_b4')

score_left_b4,count
f64,u32
0.0233,82862
0.0769,249049
0.125,86736
0.0333,648382
0.0476,558748
…,…
0.0227,45145
0.1667,44909
0.0417,623515


alt.Chart(...)

score_r8_b4,count
f64,u32
0.0588,136677
0.0333,627015
0.0476,258255
0.025,713193
0.0769,58470
…,…
0.1667,4963
0.0227,345103
0.0526,192050


alt.Chart(...)